In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images.shape, train_labels.shape

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']            

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()


In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=10)


In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)


In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
class_names[np.argmax(predictions[0])]

In [ ]:
import cv2
import numpy as np
kern_sz = (7,7)
input_sz = (28,28,1)
kernels = []
for lambd in range(1,3):
    for theta in range(0,3):
        kernel = cv2.getGaborKernel(ksize=kern_sz, sigma=1.0, theta=theta, lambd=lambd, gamma=1.0, psi=1.0)
        kernels = kernels + [[kernel]]
gabor_pyr = np.asarray(kernels)
gabor_pyr = np.transpose(gabor_pyr)
print(gabor_pyr.shape)

In [ ]:
conv_layer = keras.layers.Conv2D(name='conv_up',
                filters=gabor_pyr.shape[-1],
                kernel_size=kern_sz, 
                strides=(1,1),
                padding='same',
                use_bias=True,
                activation='relu')
conv_layer.build((None,input_sz,input_sz,1))
conv_layer.set_weights([gabor_pyr,np.zeros(conv_layer.filters)])
print(conv_layer.get_weights()[0].shape)
conv_layer.trainable = False

In [ ]:
gabor_inv = keras.Sequential()
gabor_inv.add(conv_layer)    
conv_transpose_layer = keras.layers.Conv2DTranspose(name='conv_back',
            filters=1, 
            kernel_size=kern_sz, 
            strides=(1,1), 
            padding='same',
            activation='relu')
conv_transpose_layer.trainable = True
gabor_inv.add(conv_transpose_layer)


In [ ]:
from IPython.display import SVG
from keras.utils import model_to_dot
SVG(model_to_dot(gabor_inv).create(prog='dot', format='svg'))